Author: Kaveh Mahdavi <kavehmahdavi74@yahoo.com>
License: BSD 3 clause
last update: 28/12/2022

# Representing text as Tensors

I explore different neural network architectures for dealing with natural language text by using:
* bag-of-words
* embeddings
* recurrent neural network

In [1]:
import os

os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
import tensorflow as tf
from tensorflow import keras
import tensorflow_datasets as tfds

In [2]:
# To use GPU memory cautiously, I set tensorflow option to grow GPU memory allocation when needed.
physical_devices = tf.config.list_physical_devices('GPU')
if len(physical_devices) > 0:
    tf.config.experimental.set_memory_growth(physical_devices[0], True)

## Represent text

To solve Natural Language Processing (NLP) tasks with ANN, I need some way to represent text as tensors.

* **Character-level representation:** I represent text by treating each character as a number. Given that we have C  different characters in our text corpus, the word Hello could be represented by a tensor with shape C×5. Each letter would correspond to a tensor in one-hot encoding.
*
* **Word-level representation:** I create a vocabulary of all words in our text, and then represent words using one-hot encoding. This approach is better than character-level representation because each letter by itself does not have much meaning. By using higher-level semantic concepts - words - we simplify the task for the neural network. However, given a large dictionary size, we need to deal with high-dimensional sparse tensors.

### Load Dataset

[AG_NEWS](http://www.di.unipi.it/~gulli/AG_corpus_of_news_articles.html) dataset is a collection of more than 1  million news articles. News articles have been gathered from more than
2000  news sources by ComeToMyHead in more than 1 year of activity.

In [3]:
dataset = tfds.load('ag_news_subset')

In [4]:
classes = ['World', 'Sports', 'Business', 'Sci/Tech']
ds_train = dataset['train']
ds_test = dataset['test']

print("Size of train dataset: {}".format(len(ds_train)))
print("Size of test dataset:  {}".format(len(ds_test)))

Size of train dataset: 120000
Size of test dataset:  7600


In [5]:
for i, x in zip(range(3), ds_train):
    print(f"{x['label']} ({classes[x['label']]}) -> {x['title']} {x['description']}")

3 (Sci/Tech) -> b'AMD Debuts Dual-Core Opteron Processor' b'AMD #39;s new dual-core Opteron chip is designed mainly for corporate computing applications, including databases, Web services, and financial transactions.'
1 (Sports) -> b"Wood's Suspension Upheld (Reuters)" b'Reuters - Major League Baseball\\Monday announced a decision on the appeal filed by Chicago Cubs\\pitcher Kerry Wood regarding a suspension stemming from an\\incident earlier this season.'
2 (Business) -> b'Bush reform may have blue states seeing red' b'President Bush #39;s  quot;revenue-neutral quot; tax reform needs losers to balance its winners, and people claiming the federal deduction for state and local taxes may be in administration planners #39; sights, news reports say.'


## Approaches to Represent Text as Tensor

### 1. Bag-of-Words as Data Preprocessing

I vectorize text into numbers to represent as tensors. In the word-level, I should do:
* Use a tokenizer to split text into tokens.
* Build a vocabulary of those tokens.

I don't take to account words that are rarely present in the text, since only a few sentences will have them, and the model will not learn from them.
I limit the vocabulary size by passing an argument to the `TextVectorization` constructor.

#### 1.1. Vectorize & Build a Vocabulary

In [11]:
# vocabulary size limiting
vectorizer = keras.layers.experimental.preprocessing.TextVectorization(max_tokens=50000)
vectorizer.adapt(ds_train.take(1000).map(lambda x: x['title'] + ' ' + x['description']))

vocabulary = vectorizer.get_vocabulary()
vocabulary_size = len(vocabulary)
print(vocabulary[:15])
print(f"Number of vocabulary: {vocabulary_size}")
vectorizer('I love artificial intelligence')

['', '[UNK]', 'the', 'to', 'a', 'in', 'of', 'and', 'on', 'for', '39s', 'that', 'at', 'as', 'with']
Number of vocabulary: 8317


<tf.Tensor: shape=(4,), dtype=int64, numpy=array([ 178, 5930, 7893, 6226])>

#### 1.2. Bagging

I convert each word number into a one-hot encoding and adding all those vectors up.

In [14]:
def get_bag_of_words(text, vocab_size):
    return tf.reduce_sum(tf.one_hot(vectorizer(text), vocab_size), axis=0)


batch_size = 128
ds_train_bow = ds_train.map(lambda x: (get_bag_of_words(x['title'] + x['description'], vocabulary_size),
                                       x['label'])).batch(batch_size)
ds_test_bow = ds_test.map(lambda x: (get_bag_of_words(x['title'] + x['description'], vocabulary_size),
                                     x['label'])).batch(batch_size)

#### 1.3. Build Classifier

In [94]:
model = keras.models.Sequential([
    keras.layers.Dense(4, activation='softmax', input_shape=(vocabulary_size,))
])

model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['acc'])
model.fit(ds_train_bow, validation_data=ds_test_bow)

model.summary()

938/938 [==============================] - 68s 72ms/step - loss: 0.6164 - acc: 0.8423 - val_loss: 0.4418 - val_acc: 0.8688
Model: "sequential_21"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_18 (Dense)            (None, 4)                 21344     
                                                                 
Total params: 21,344
Trainable params: 21,344
Non-trainable params: 0
_________________________________________________________________


### 2. Bag-of-Words with n-grams

Since some words are part of multi-word expressions, for example, the word 'on-line' has a completely different meaning.

from the words 'on' and 'line' in other contexts. so the representation of 'on' and 'line' by the same vectors, it can confuse our model.

IN n-gram the frequency of each word, bi-word or tri-word is a useful feature for training classifiers, e.g. bigram
adds all word pairs to the vocabulary, in addition to original words.

#### 2.1 Generate a bi-gram Bag-of-Words

To use an n-gram representation in our AG News dataset, we need to pass the ngrams parameter to our TextVectorization constructor.

In [12]:
vectorizer = keras.layers.experimental.preprocessing.TextVectorization(max_tokens=50000, ngrams=2)
vectorizer.adapt(ds_train.take(500).map(lambda x: x['title'] + ' ' + x['description']))

vocabulary = vectorizer.get_vocabulary()
vocabulary_size = len(vocabulary)
print(vocabulary[:15])
print(f"Number of vocabulary: {vocabulary_size}")
vectorizer('I love artificial intelligence')

['', '[UNK]', 'the', 'to', 'a', 'in', 'of', 'and', 'on', 'for', '39s', 'with', 'that', 'its', 'as']
Number of vocabulary: 20274


<tf.Tensor: shape=(7,), dtype=int64, numpy=array([  130, 11718, 18382, 12901,     1,     1, 18381])>

#### 2.2. Bagging

In [19]:
ds_train_bow_ngram = ds_train.map(lambda x: (get_bag_of_words(x['title'] + x['description'], vocabulary_size),
                                             x['label'])).batch(batch_size)
ds_test_bow_ngram = ds_test.map(lambda x: (get_bag_of_words(x['title'] + x['description'], vocabulary_size),
                                           x['label'])).batch(batch_size)

#### 2.3. Build Classifier

In [16]:
model = keras.models.Sequential([
    keras.layers.Dense(4, activation='softmax', input_shape=(vocabulary_size,))
])

model.summary()

model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['acc'])
model.fit(ds_train_bow_ngram, validation_data=ds_test_bow_ngram)

938/938 [==============================] - 347s 370ms/step - loss: 0.5827 - acc: 0.8484 - val_loss: 0.4239 - val_acc: 0.8707
Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 4)                 81100     
                                                                 
Total params: 81,100
Trainable params: 81,100
Non-trainable params: 0
_________________________________________________________________


### 3. Bag-of-Words as a Layer

Since the vectorizer is also a Keras layer, I can define a network that includes it, and train it end-to-end.

Then I don't need to vectorize the dataset using map, we can just pass the original dataset to the input of the network.


In [20]:
# Vectorize Text
vectorizer_layer = keras.layers.experimental.preprocessing.TextVectorization(max_tokens=50000)
vectorizer_layer.adapt(ds_train.take(500).map(lambda x: x['title'] + ' ' + x['description']))

vocabulary = vectorizer_layer.get_vocabulary()
vocabulary_size = len(vocabulary)

In [21]:
def to_tuple(_x):
    return _x['title'] + ' ' + _x['description'], _x['label']

In [ ]:
batch_size = 128
ds_train_embed = ds_train.map(to_tuple).batch(batch_size)
ds_test_embed = ds_test.map(to_tuple).batch(batch_size)

inp = keras.Input(shape=(1,), dtype=tf.string)
x = vectorizer_layer(inp)
x = tf.reduce_sum(tf.one_hot(x, vocabulary_size), axis=1)
out = keras.layers.Dense(4, activation='softmax')(x)
model = keras.models.Model(inp, out)

model.summary()

model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['acc'])
model.fit(ds_train_embed, validation_data=ds_test_embed)

### 4. Auto-compute Bag-of-Words

Until here, I compute the BoW vectors manually by summing the one-hot encodings of individual words to show the calculation approach.

To define and train the model easier, TensorFlow enable us to calculate BoW vectors automatically by passing the  `output_mode='count parameter'` to the vectorizer constructor.


In [20]:
model = keras.models.Sequential([
    keras.layers.experimental.preprocessing.TextVectorization(max_tokens=vocabulary_size, output_mode='count'),
    keras.layers.Dense(4, input_shape=(vocabulary_size,), activation='softmax')
])
print("Training vectorizer:")
model.layers[0].adapt(ds_train.take(500).map(lambda x: x['title'] + ' ' + x['description']))
model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['acc'])
model.fit(ds_train_embed, validation_data=ds_test_embed)

model.summary()

Training vectorizer:
938/938 [==============================] - 10s 10ms/step - loss: 0.5920 - acc: 0.8489 - val_loss: 0.4166 - val_acc: 0.8772
Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 text_vectorization_6 (TextV  (None, 5334)             0         
 ectorization)                                                   
                                                                 
 dense_4 (Dense)             (None, 4)                 21340     
                                                                 
Total params: 21,340
Trainable params: 21,340
Non-trainable params: 0
_________________________________________________________________


### 5. Term frequency - Inverse Document Frequency

It's a variation of bag-of-words, where instead of a binary 0/1 value indicating the appearance of a word in a document, a floating-point value is used, which is related to the frequency of the word occurrence in the corpus.

The weight $w_{ij}$ of a word $i$ in the document $j$ is defined as:
$$
w_{ij} = tf_{ij}\times\log({N\over df_i})
$$
where
* $tf_{ij}$ is the number of occurrences of $i$ in $j$, i.e. the BoW value we have seen before
* $N$ is the number of documents in the collection
* $df_i$ is the number of documents containing the word $i$ in the whole collection

The TF-IDF value wij increases proportionally to the number of times a word appears in a document and is offset by the number of documents in the corpus that contains the word, which helps to adjust for the fact that some words appear more frequently than others. For example, if the word appears in every document in the collection, dfi=N, and wij=0, and those terms would be completely disregarded.

In [24]:
batch_size = 128
ds_train_embed = ds_train.map(to_tuple).batch(batch_size)
ds_test_embed = ds_test.map(to_tuple).batch(batch_size)

model = keras.models.Sequential([
    keras.layers.experimental.preprocessing.TextVectorization(max_tokens=vocabulary_size,output_mode='tf-idf'),
    keras.layers.Dense(4,input_shape=(vocabulary_size,), activation='softmax')
])
print("Training vectorizer")
model.layers[0].adapt(ds_train.take(500).map(lambda x: x['title'] + ' ' + x['description']))

model.compile(loss='sparse_categorical_crossentropy',optimizer='adam',metrics=['acc'])
model.fit(ds_train_embed,validation_data=ds_test_embed)

model.summary()

Training vectorizer
938/938 [==============================] - 15s 15ms/step - loss: 0.4173 - acc: 0.8681 - val_loss: 0.3410 - val_acc: 0.8883
Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 text_vectorization_10 (Text  (None, 5334)             1         
 Vectorization)                                                  
                                                                 
 dense_3 (Dense)             (None, 4)                 21340     
                                                                 
Total params: 21,341
Trainable params: 21,340
Non-trainable params: 1
_________________________________________________________________
